In [ ]:
import numpy as np
from MetaLearner import *
import pickle

In [ ]:
# todo: https://stackoverflow.com/questions/24168569/how-to-add-names-to-a-numpy-array-without-changing-its-dimension

In [ ]:
# This file runs the experiment

In [ ]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic.pkl"
open_file = open(file_name, "rb")
data = pickle.load(open_file)
open_file.close()

In [ ]:
n_setups = 2
sample_sizes = [200, 200]
n_runs = 2
learners = [TLearner('rf'), SLearner('rf'), XLearner('rf'), RLearner('rf'), DRLearner('rf'), RALearner('rf'),
            PWLearner('rf'), ULearner('rf'),
            TLearner('lasso'), SLearner('lasso'), XLearner('lasso'), RLearner('lasso'), DRLearner('lasso'),
            RALearner('lasso'), PWLearner('lasso'), ULearner('lasso'),
            TLearner('nn'), SLearner('nn'), XLearner('nn'), RLearner('nn'), DRLearner('nn'), RALearner('nn'),
            PWLearner('nn'), ULearner('nn')]


In [ ]:
# What we want: For each setting and base learner --> matrix which includes the MSE, columns: Metalearners + SampleSize, rows: Run(1-10)
# --> So we can plot the mean and 1se of the MSE for each Metalearner, each samplesize; giving the setting and baselearner
# --> number of plots: settings*baselearner.

In [ ]:
results = []

for i in range(n_setups):
    results.append([])

for i in range(n_setups):
    for baselearner in range(3):
        results[i].append([])

results

In [ ]:
dim = data[0][0][0][0].shape[1]  # dimension of x


# helper function to get y, x, w, tau, out of data[][][][]
def get_variables(dataset, setup, samplesize, run, train_test):
    y = dataset[setup][samplesize][run][train_test][:, 0]
    x = dataset[setup][samplesize][run][train_test][:, 1:(dim - 2)]
    w = dataset[setup][samplesize][run][train_test][:, (dim - 2)]
    tau = dataset[setup][samplesize][run][train_test][:, (dim - 1)]
    return y, x, w, tau

In [ ]:
b = 0
m = 0
s = 0

for i in range(n_setups):
    print(f'Setup: {i + 1}')
    # array for all mses for one setup
    setup_mse = np.empty(shape=(0, 24))
    s = 0  # restart index for samplesize
    for size in sample_sizes:
        print(f'Sample Size: {s + 1}')
        # array for all mses for one setup and samplesize.
        size_mse = np.empty(shape=(0, 24))
        for r in range(n_runs):
            print(f'Run: {r + 1}')
            # array for all mses in one setup, samplesize and run.
            mses = np.empty(shape=(1, 24))
            # get data for specific setup, samplesize and run.
            temp_y_train, temp_x_train, temp_w_train, temp_tau_train = get_variables(dataset=data, setup=i,
                                                                                     samplesize=s, run=r,
                                                                                     train_test=0)
            temp_y_test, temp_x_test, temp_w_test, temp_tau_test = get_variables(dataset=data, setup=i,
                                                                                 samplesize=s, run=r,
                                                                                 train_test=1)
            # restart index for metalearner
            m = 0
            for l in learners:
                if m % 8 == 0: print(f'BaseLearner {m / 8 + 1}')
                # training and testing MetaLearner.
                learner = l
                learner.fit(temp_x_train, temp_y_train, temp_w_train)
                predictions = learner.predict(temp_x_test)
                temp_mse = ((predictions - temp_tau_test) ** 2).mean()
                # append mse of specific metalearner to 'mses'.
                mses[0, m] = temp_mse
                # update index
                m += 1
            # append 'mses' to 'size_mse'.
            size_mse = np.append(size_mse, mses, axis=0)
        # append 'size_mse' to 'setup_mse'.
        setup_mse = np.append(setup_mse, size_mse, axis=0)
        # update index
        s += 1
    # append to results
    results[i][0] = setup_mse[:, 0:8]
    results[i][1] = setup_mse[:, 8:16]
    results[i][2] = setup_mse[:, 16:24]





# this should make sense i guess! :)
# TODO: CHECK IF IT MAKES IT FASTER IF EVERYTHING IS EITHER IN NP OR IN LIST/ARRAY BASE PYTHON.
# TODO: DONE :)
# TODO: DONE :)
# TODO: DONE :)

### IT TOOK 6M 40S FOR --> SAMPLE SIZES [200, 200], TEST SIZE = 50, 2 SETUPS, 2 RUNS LOL

In [ ]:
results[1][2]  # NICE :) (results probably bad, cuz sample size = 200 lol)

small check

In [ ]:
mses

In [ ]:
size_mse

In [ ]:
setup_mse

In [ ]:
# NICE :)))